In [1]:
import numpy as np
import pandas as pd
import glob
import os
from importlib import reload

from sklearn.metrics.pairwise import haversine_distances

In [24]:
src = '../results/'
filename = src + 'author_coauthor_corresponding.csv'
data = pd.read_csv(filename)
print(data.shape)
data.head(15)

(292520, 3)


,coauthor_num,corresponding_num,corr_institutions
0,3,1,0
1,2,1,0
2,4,1,0
3,6,1,0
4,3,1,0
5,2,1,0
6,3,1,0
7,4,1,0
8,3,1,0
9,2,1,1


In [29]:
np.asarray(data.corr_institutions[14].split(','), dtype=int)

array([0])

In [57]:
numcorr = np.sum(data.corresponding_num.values)
numcorr

331031

In [11]:
filename = src + 'split_locations_of_all_authors_20220505.csv'
df = pd.read_csv(filename)
print(df.shape)
df.head()

(777681, 9)


,original_id,full_name,city,state,country,zip_code,location,latitude,longitude
0,0,"Univ Autonoma Madrid, Dept Biol Mol, E-28049 M...",Madrid,NaN,Spain,E-28049,"Madrid, Mostoles, Spain",40.323213,-3.867629
1,0,"IEO, Ctr Oceanog Vigo, Vigo 36390, Spain",Vigo,NaN,Spain,36390,"Vigo, Pontevedra, Spain",42.240599,-8.720727
2,0,"MIT, Dept Civil & Environm Engn, 77 Massachuse...",Cambridge,MA,USA,02139,"Cambridge, MA, USA",42.373616,-71.109733
3,1,"Assiut Univ, Fac Sci, Dept Bot & Microbiol, As...",Assiut,NaN,Egypt,71516,"Assiut, Egypt",27.178312,31.185926
4,1,"Assiut Univ, Fac Agr, Dept Plant Pathol, Assiu...",Assiut,NaN,Egypt,71526,"Assiut, Egypt",27.178312,31.185926


In [31]:
corresponding_authors = [None for i in range(numcorr)]

In [51]:
idx = 0
jdx = 0
for i in range(len(data)):
    coauth_num = data.coauthor_num[i]
    corr = np.asarray(data.corr_institutions[i].split(','), dtype=int)
    aux = df.iloc[idx:idx+coauth_num, :]
    for j in corr:
        corresponding_authors[jdx] = aux.iloc[j,:]
        jdx += 1
    idx += coauth_num

In [55]:
corr = pd.concat(corresponding_authors, axis=1).T

In [150]:
corr[['latitude', 'longitude']] = corr[['latitude', 'longitude']].apply(pd.to_numeric)
#corr.iloc[:, 7:] = pd.to_numeric(corr.iloc[:, 7:])
corr

,original_id,full_name,city,state,country,zip_code,location,latitude,longitude
0,0,"Univ Autonoma Madrid, Dept Biol Mol, E-28049 M...",Madrid,NaN,Spain,E-28049,"Madrid, Mostoles, Spain",40.323213,-3.867629
3,1,"Assiut Univ, Fac Sci, Dept Bot & Microbiol, As...",Assiut,NaN,Egypt,71516,"Assiut, Egypt",27.178312,31.185926
5,2,"Sapienza Univ Roma, Dipartimento Biol Ambienta...",Rome,NaN,Italy,I-00185,"Rome, RM, Italy",41.902784,12.496365
9,3,"Fac Biol, Dept Plant Biol, Lab Algol, Espinard...",Murcia,NaN,Spain,E-30100,"Murcia, Espinardo, Spain",38.011190,-1.155696
15,4,"Univ Murcia, Fac Biol, Dept Biol Vegetal, Lab ...",Murcia,NaN,Spain,E-30100,"Murcia, Espinardo, Spain",38.011190,-1.155696
...,...,...,...,...,...,...,...,...,...
777667,292515,"USDA ARS, Organ Res Program, Salinas, CA 93905...",Salinas,CA,USA,93905,"Salinas, CA, USA",36.677737,-121.655501
777669,292516,"Kansas State Univ, Dept Agron, Throckmorton Pl...",Manhattan,KS,USA,66506,"Manhattan, KS, USA",39.183608,-96.571669
777671,292517,"Dept Agron & Hort, N Platte, NE 69101 USA",N Platte,NE,USA,69101,"North Platte, NE, USA",41.140276,-100.760145
777675,292518,"Univ Missouri, Dept Agron, Novelty, MO 63460 USA",Novelty,MO,USA,63460,"Novelty, MO, USA",40.012539,-92.208238


In [152]:
print(corr.shape)
filename = src + 'split_locations_of_corr_authors_20220505.csv'
corr.to_csv(filename, index=False)

(331031, 9)


# Tally each location

In [153]:
countries = np.unique(corr.country)
countriesdf = [None for i in range(len(countries))]

In [170]:
country = 'Mexico'
country = 'Afghanistan'

cdf = corr[corr.country == country]

locs = cdf.location.values
uq_locs, idxs= np.unique(locs, return_index = True)
locs_dict = dict(zip(uq_locs, np.zeros(len(uq_locs), dtype=int)))
for key in locs:
    locs_dict[key] += 1

scores = np.asarray(list(locs_dict.values()))
argsort = np.argsort(scores)[::-1]
#name_dict = dict(zip(uq_locs[argsort], foo[argsort]))

In [171]:
argsort

array([0])

In [172]:
uqcdf = cdf.iloc[idxs[argsort], [0,4,6,7,8]].copy()
uqcdf.columns = ['incountry_rank'] + list(uqcdf.columns[1:])
uqcdf['score'] = scores[argsort]
uqcdf = uqcdf.iloc[:, [0,1,2,5,3,4]]

print(uqcdf.shape)
uqcdf.head(15)

(1, 6)


,incountry_rank,country,location,score,latitude,longitude
53378,23200,Afghanistan,"Kabul, Afghanistan",1,34.555349,69.207486


In [157]:
locs = np.deg2rad(uqcdf.iloc[:, 4:].values)
dists = haversine_distances(locs)
dists *= 6371000/1000

In [159]:
tol = 8.0
corresp = np.column_stack([np.ones(len(uqcdf)).astype(int), 
                           np.arange(len(uqcdf)), 
                           uqcdf.score.values])

for idx in range(1, len(corresp)):
    for j in range(idx):
        if dists[idx, j] < tol:
            k = corresp[j, 1]
            corresp[idx, 0] = 0
            corresp[idx,1] = k
            corresp[k,2] += corresp[idx, 2]
            
            print(idx,'\t',uqcdf.iloc[idx,2], '-->', uqcdf.iloc[k,2], '\t(', np.around(dists[idx,k],1), ')')
            break

9 	 Ciudad De Mexico, CDMX, Mexico --> Mexico City, CDMX, Mexico 	( 0.0 )
16 	 Texcoco, Edo De Mexico, Mexico --> Texcoco, Estado De Mexic, Mexico 	( 0.0 )
18 	 El Batan, Texcoco, Mexico --> Texcoco, Estado De Mexic, Mexico 	( 3.7 )
23 	 San Luis Potosi, Slp, Mexico --> San Luis Potosi, San Luis Potosi, Mexico 	( 0.0 )
24 	 Guadalajara, Jalisco, Mexico --> Zapopan, Jalisco, Mexico 	( 7.1 )
28 	 Montecillo, Edo De Mexico, Mexico --> Texcoco, Estado De Mexic, Mexico 	( 6.4 )
31 	 La Paz, Bcs, Mexico --> La Paz, Baja California Sur, Mexico 	( 0.0 )
33 	 Queretaro, Qro, Mexico --> Queretaro, Queretaro, Mexico 	( 0.0 )
34 	 Monterrey, Nuevo Leon, Mexico --> Monterrey, NL, Mexico 	( 0.0 )
41 	 La Paz, Baja Calif Sur, Mexico --> La Paz, Baja California Sur, Mexico 	( 0.0 )
42 	 La Paz, BCS, Mexico --> La Paz, Baja California Sur, Mexico 	( 0.0 )
45 	 El Marques, Queretaro, Mexico --> Queretaro, Queretaro, Mexico 	( 6.8 )
48 	 Chapingo, Edo De Mexico, Mexico --> Texcoco, Estado De Mexic, Mexic

In [160]:
mask = np.nonzero(corresp[:,0])[0]
dfred = uqcdf.iloc[mask,:]
dfred = dfred.assign(score=corresp[mask, 2])
dfred = dfred.sort_values(by='score', ascending=False)
dfred = dfred.assign(incountry_rank=np.arange(len(dfred)))
dfred.head(15)

,incountry_rank,country,location,score,latitude,longitude
731,0,Mexico,"Mexico City, CDMX, Mexico",1016,19.432608,-99.133208
34524,1,Mexico,"Cuernavaca, Morelos, Mexico",291,18.924209,-99.221566
7880,2,Mexico,"Merida, Yucatan, Mexico",277,20.967370,-89.592586
25658,3,Mexico,"Irapuato, Guanajuato, Mexico",233,20.678665,-101.354496
18619,4,Mexico,"Morelia, Michoacan, Mexico",217,19.705950,-101.194982
52972,5,Mexico,"Texcoco, Estado De Mexic, Mexico",217,19.506038,-98.883150
4589,6,Mexico,"Xalapa, Veracruz, Mexico",173,19.543775,-96.910181
9788,7,Mexico,"San Luis Potosi, San Luis Potosi, Mexico",121,22.156470,-100.985541
32278,8,Mexico,"Saltillo, Coahuila, Mexico",78,25.438323,-100.973678
11376,9,Mexico,"Monterrey, NL, Mexico",66,25.686614,-100.316113


# Concatenating everything

In [163]:
country

'Afghanistan'

In [173]:
cdx = 0
for country in countries:
    
    cdf = corr[corr.country == country]

    locs = cdf.location.values
    uq_locs, idxs= np.unique(locs, return_index = True)
    locs_dict = dict(zip(uq_locs, np.zeros(len(uq_locs), dtype=int)))
    for key in locs:
        locs_dict[key] += 1

    scores = np.asarray(list(locs_dict.values()))
    argsort = np.argsort(scores)[::-1]
    
    uqcdf = cdf.iloc[idxs[argsort], [0,4,6,7,8]].copy()
    uqcdf.columns = ['incountry_rank'] + list(uqcdf.columns[1:])
    uqcdf['score'] = scores[argsort]
    uqcdf = uqcdf.iloc[:, [0,1,2,5,3,4]]
    
    locs = np.deg2rad(uqcdf.iloc[:, 4:].values)
    dists = haversine_distances(locs)
    dists *= 6371000/1000
    
    corresp = np.column_stack([np.ones(len(uqcdf)).astype(int), 
                               np.arange(len(uqcdf)), 
                               uqcdf.score.values])

    for idx in range(1, len(corresp)):
        for j in range(idx):
            if dists[idx, j] < tol:
                k = corresp[j, 1]
                corresp[idx, 0] = 0
                corresp[idx,1] = k
                corresp[k,2] += corresp[idx, 2]
                break

    mask = np.nonzero(corresp[:,0])[0]
    dfred = uqcdf.iloc[mask,:]
    dfred = dfred.assign(score=corresp[mask, 2])
    dfred = dfred.sort_values(by='score', ascending=False)
    dfred = dfred.assign(incountry_rank=np.arange(len(dfred)))
    
    countriesdf[cdx] = dfred
    cdx += 1

In [175]:
dfred = pd.concat(countriesdf, ignore_index=True)
print(dfred.shape)

(5606, 6)


In [177]:
filename = src + 'merged_corresponding_locations.csv'
dfred.to_csv(filename, index=False)
dfred.head()

,incountry_rank,country,location,score,latitude,longitude
0,0,Afghanistan,"Kabul, Afghanistan",1,34.555349,69.207486
1,0,Albania,"Tirana, Albania",12,41.327546,19.818698
2,0,Algeria,"Algiers, Algeria",109,36.753768,3.058756
3,1,Algeria,"Constantine, Algeria",41,36.357005,6.639028
4,2,Algeria,"Batna, Algeria",18,35.563419,6.189000
